# Setup

## Import statements

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.7.0


## Set seed

In [ ]:
# Random seed for reproducibility
seed = 69

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Load dataset

In [ ]:
dataset = pd.read_csv('Training.csv')
target_labels = dataset.columns
print(dataset.shape)
print(target_labels)
dataset.head()

(68528, 7)
Index(['Sponginess', 'Wonder level', 'Crunchiness', 'Loudness on impact',
       'Meme creativity', 'Soap slipperiness', 'Hype root'],
      dtype='object')


,Sponginess,Wonder level,Crunchiness,Loudness on impact,Meme creativity,Soap slipperiness,Hype root
0,7.97698,4.33494,10.67282,1.76692,3.22440,51.681460,3.65434
1,8.07824,4.44616,10.56160,1.70716,3.32566,51.563598,3.47672
2,8.02844,4.22372,10.56160,1.64906,3.17460,50.863080,3.47672
3,8.02844,4.22372,10.56160,1.70716,3.17460,45.841581,3.47672
4,7.87572,4.44616,10.45038,1.70716,3.27586,47.126421,3.47672


## Hyperparameters


In [ ]:
model_name = "H_MK_XIX"
window = 2000
stride = 10
telescope = 16
reg_telescope = 864
epochs = 200
batch_size = 64 #sembra ottimale un batch size piccolo es 8
# layer_size = 128
# regularization = 1e-6
# kernel_size = 3
# dropout_value1 = 0.5
# dropout_value2 = 0.5
# clipnorm=0.01

# Data Preparation

## Segment dataset

In [ ]:
test_size = int(dataset.shape[0]/10)
X_train_raw = dataset.iloc[:-test_size]
X_train_window = dataset.iloc[:window]
X_test_raw = dataset.iloc[-test_size:]
print(X_train_raw.shape, X_test_raw.shape)

(61676, 7) (6852, 7)


## Normalize

In [ ]:
X_min = dataset.min()
X_max = dataset.max()

X_train_raw = (X_train_raw-X_min)/(X_max-X_min)
X_train_window = (X_train_window-X_min)/(X_max-X_min)
X_test_raw = (X_test_raw-X_min)/(X_max-X_min)

# future = dataset[-window:]
# future = (future-X_min)/(X_max-X_min)
# future = np.expand_dims(future, axis=0)
# future.shape

## Plot training/test set

In [ ]:
# def inspect_dataframe(train, window, test, columns):
#     figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(30,30))
#     for i, col in enumerate(columns):
#         axs[i].plot(train[col], label='Train')
#         axs[i].plot(window[col], label='Window')
#         axs[i].plot(test[col], label='Test')
#         axs[i].set_title(col)
#         axs[i].legend()
#     plt.show()
# inspect_dataframe(X_train_raw, X_train_window, X_test_raw, target_labels)

## Split into windows

In [ ]:
def build_sequences(df, target_labels=['sponginess'], window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_test, y_test = build_sequences(X_test_raw, target_labels, window, stride, telescope)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

print(input_shape, output_shape)

(5999, 2000, 7) (5999, 16, 7) (599, 2000, 7) (599, 16, 7)
(2000, 7) (16, 7)


# Model

## Callbacks

In [ ]:
def create_folders_and_callbacks(model_name):

  ############ Create directory for checkpoints ###########
  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  ############ Model checkpoints #################
  ckpt_dir = os.path.join(exp_dir, 'ckpts') # ckpts folder will contain model checkpoints
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), # where to save the model
                                                     save_weights_only=False, # True to save only weights (not the whole model)
                                                     save_best_only=False) # True to save only the best epoch (it's probably best to save it later manually, so you have bothe the best and LAST epoch - you can use the last to restart training if something goes wrong)
  callbacks.append(ckpt_callback)

  ############# Early Stopping #################
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  ############# Reduce Learning Rate ##############
  rlr_callback = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.7, min_lr=1e-5)
  callbacks.append(rlr_callback)

  return callbacks

## Build the model

In [ ]:
def build_model(hp):

    layer_size = hp.Int('layer', 8, 2048, default=128)
    regularization = hp.Float('regularization', 1e-8, 1e-4, sampling="log", default=1e-6)
    kernel_size = hp.Int('kernel', 1, 17, default=3)
    dropout_value1 = hp.Float('dropout1', 0, 0.5, step=0.1, default=0.5)
    dropout_value2 = hp.Float('dropout2', 0, 0.5, step=0.1, default=0.5)
    clipnorm = hp.Float('clipnorm', 1e-3, 1e-1, sampling="log", default=1e-2)

# Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')
    conv = tfkl.Conv1D(layer_size, kernel_size, padding='same', activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed),kernel_regularizer=tf.keras.regularizers.l2(regularization))(input_layer)
    conv = tfkl.MaxPool1D()(conv)
    blstm = tfkl.Bidirectional(tfkl.LSTM(layer_size, return_sequences=False,kernel_initializer=tfk.initializers.GlorotUniform(seed),kernel_regularizer=tf.keras.regularizers.l2(regularization)))(conv)
    blstm = tfkl.Dropout(dropout_value1, seed = seed)(blstm)
    dense1 = tfkl.Dense(layer_size, activation = "relu", kernel_initializer=tfk.initializers.GlorotUniform(seed),kernel_regularizer=tf.keras.regularizers.l2(regularization))(blstm)
    dense1 = tfkl.Dropout(dropout_value2, seed = seed)(dense1)
    # In order to predict the next values for more than one sensor,
    # we can use a Dense layer with a number given by telescope*num_sensors,
    # followed by a Reshape layer to obtain a tensor of dimension
    # [None, telescope, num_sensors]
    dense = tfkl.Dense(output_shape[-1] * output_shape[-2], activation='relu',kernel_initializer=tfk.initializers.GlorotUniform(seed),kernel_regularizer=tf.keras.regularizers.l2(regularization))(dense1)
    output_layer = tfkl.Reshape((output_shape[-2], output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(clipnorm=clipnorm), metrics=tf.keras.metrics.RootMeanSquaredError())

    # Return the model
    return model

In [ ]:
%pip install keras_tuner
import keras_tuner as kt


# tuner = kt.Hyperband(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_epochs=2,
#     factor=3,
#     hyperband_iterations=1,
#     distribution_strategy=tf.distribute.MirroredStrategy(),
#     directory="results_dir",
#     project_name="mnist",
#     overwrite=True,
# )

tuner = kt.BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=50,
    seed=seed,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory="results_dir",
    project_name=model_name,
    overwrite=True,
)

     |████████████████████████████████| 98 kB 3.5 MB/s 


In [ ]:
# model = build_model(input_shape, output_shape)
# model.summary()
#tfk.utils.plot_model(model, expand_nested=True)

## Train the model

In [ ]:
callbacks = create_folders_and_callbacks(model_name=model_name) 

tuner.search(x = X_train,
             y = y_train,
             batch_size = batch_size,
             epochs = epochs,
             validation_split=.1,
             callbacks=callbacks)

model = tuner.get_best_models(1)[0]

hyperparameters = tuner.get_best_hyperparameters(1)[0]
print("Layer: ", hyperparameters.get('layer'))
print("Regularization: ", hyperparameters.get('regularization'))
print("Kernel: ", hyperparameters.get('kernel'))
print("Dropout1: ", hyperparameters.get('dropout1'))
print("Dropout2: ", hyperparameters.get('dropout2'))
print("Clipnorm: ", hyperparameters.get('clipnorm'))

# history = model.fit(
#     x = X_train,
#     y = y_train,
#     batch_size = batch_size,
#     epochs = epochs,
#     validation_split=.1,
# ).history


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
layer             |1404              |?                 
regularization    |4.3559e-05        |?                 
kernel            |6                 |?                 
dropout1          |0                 |?                 
dropout2          |0.1               |?                 
clipnorm          |0.052415          |?                 

Epoch 1/200


## Save the model

In [ ]:
# model.save(model_name)
# model = tfk.models.load_model(model_name)

## Plot the results

In [ ]:
# best_epoch = np.argmin(history['val_loss'])
# plt.figure(figsize=(17,4))
# plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
# plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
# plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
# plt.title('Mean Squared Error (Loss)')
# plt.legend()
# plt.grid(alpha=.3)
# plt.show()

In [ ]:
# plt.figure(figsize=(17,4))
# plt.plot(history['rmse'], label='Training accuracy', alpha=.8, color='#ff7f0e')
# plt.plot(history['val_rmse'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
# plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
# plt.title('Root Mean Squared Error')
# plt.legend()
# plt.grid(alpha=.3)
# plt.show()

In [ ]:
# plt.figure(figsize=(17,4))
# plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
# plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
# plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
# plt.title('Mean Absolute Error')
# plt.legend()
# plt.grid(alpha=.3)
# plt.show()

In [ ]:
# plt.figure(figsize=(18,3))
# plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
# plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
# plt.legend()
# plt.grid(alpha=.3)
# plt.show()

# Test

## Setup

In [ ]:
predictions = model.predict(X_test)
print(predictions.shape)

mean_squared_error = tfk.metrics.mse(y_test.flatten(),predictions.flatten())
mean_absolute_error = tfk.metrics.mae(y_test.flatten(),predictions.flatten())
print(mean_squared_error, mean_absolute_error)

In [ ]:
def inspect_multivariate_prediction(X, y, pred, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].plot(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].plot(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), pred[idx,:,i], color='green')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

## Print

In [ ]:
inspect_multivariate_prediction(X_test, y_test, predictions, target_labels, telescope)

In [ ]:

X_test_reg, y_test_reg = build_sequences(X_test_raw, target_labels, window, stride, reg_telescope)
print(X_test_reg.shape, y_test_reg.shape)

reg_predictions = np.array([])
X_temp = X_test_reg
for reg in range(int(reg_telescope/telescope)):
    pred_temp = model.predict(X_temp)
    if(len(reg_predictions)==0):
        reg_predictions = pred_temp
    else:
        reg_predictions = np.concatenate((reg_predictions,pred_temp),axis=1)
    X_temp = np.concatenate((X_temp[:,telescope:,:],pred_temp), axis=1)



#lo uso col direct
'''
reg_telescope = telescope
X_test_reg, y_test_reg = build_sequences(X_test_raw, target_labels, window, stride, reg_telescope)
predictions = model.predict(X_test)
reg_predictions = predictions
'''


In [ ]:
#parte in comune

print(reg_predictions.shape)

mean_squared_error = tfk.metrics.mse(y_test_reg.flatten(),reg_predictions.flatten())
mean_absolute_error = tfk.metrics.mae(y_test_reg.flatten(),reg_predictions.flatten())
print(mean_squared_error, mean_absolute_error)

inspect_multivariate_prediction(X_test_reg, y_test_reg, reg_predictions, target_labels, reg_telescope)

